In [6]:
import pandas as pd
import re

# --- 1. Función para limpiar texto ---
def limpiar_texto(texto):
    if pd.isna(texto):
        return ""
    texto = str(texto).replace('\n', ' ').replace('\r', ' ')
    texto = re.sub(r'\s+', ' ', texto)
    return texto.strip()

# --- 2. Cargar dataset traducido ---
df_es = pd.read_csv('../data/translated/worldnews_balanced_es.csv')
df_es = df_es.rename(columns={'title_es': 'title', 'text_es': 'text'})
df_es = df_es[['title', 'text', 'label']]

# --- 3. Cargar test.csv ---
df_test_raw = pd.read_csv(
    '../data/raw/test.csv',
    quoting=1,
    quotechar='"',
    encoding='utf-8',
    engine='python'
)

# --- 4. Estandarizar columnas ---
df_test = df_test_raw.rename(columns={
    'HEADLINE': 'title',
    'TEXT': 'text',
    'CATEGORY': 'label'
})

# Mapear etiquetas 'TRUE'/'FALSE' → 'real'/'fake'
df_test['label'] = df_test['label'].astype(str).str.strip().str.upper()
df_test['label'] = df_test['label'].map({'TRUE': 'real', 'FALSE': 'fake'})

# Guardar filas inválidas si las hay
filas_invalidas = df_test[df_test['label'].isna()]
if not filas_invalidas.empty:
    print(f"❌ Filas sin etiqueta válida: {len(filas_invalidas)}")
    filas_invalidas.to_csv('../data/processed/filas_con_label_invalido.csv', index=False)

df_test = df_test[['title', 'text', 'label']]

# --- 5. Unificar datasets ---
df_total = pd.concat([df_es, df_test], ignore_index=True)

# --- 6. Limpiar y unificar en una sola columna ---
df_total['title'] = df_total['title'].apply(limpiar_texto)
df_total['text'] = df_total['text'].apply(limpiar_texto)

# ✅ Crear columna unificada "contenido"
df_total['contenido'] = (df_total['title'] + '. ' + df_total['text']).str.strip()

# Si por alguna razón title está vacío, solo deja el texto
df_total['contenido'] = df_total.apply(
    lambda row: row['text'] if row['title'] == "" else row['contenido'],
    axis=1
)

# --- 7. Eliminar columnas innecesarias ---
df_total = df_total[['contenido', 'label']]

# --- 8. Guardar resultado ---
df_total.to_csv('../data/processed/dataset_unificado.csv', index=False)

# --- 9. Reporte ---
print("✅ Dataset unificado creado.")
print(f"Total de registros combinados: {len(df_total)}")
print("Distribución de etiquetas:")
print(df_total['label'].value_counts())


✅ Dataset unificado creado.
Total de registros combinados: 1072
Distribución de etiquetas:
label
fake    536
real    536
Name: count, dtype: int64
